In [62]:
import maya.cmds as cmds
import maya.api.OpenMaya as om

In [63]:
%setPort 4434

In [109]:
fnTrans = om.MFnTransform()
trans1 = fnTrans.create()
print(fnTrans.name())
print(fnTrans.object() == trans1)

transform4
True


In [168]:
trans2 = fnTrans.create()
print(fnTrans.name())

transform10


In [266]:
print(om.MFnDagNode(fnTrans.parent(0)).name())

world


In [202]:
def printParent(obj: om.MObject):
    if obj.hasFn(om.MFn.kDagNode):
        obj = om.MFnDagNode(obj).parent(0)
    else:
        print(f"{om.MFnDependencyNode(obj).name()} is not a dag node")
        return
    while obj.hasFn(om.MFn.kDagNode):
        fnDagNode = om.MFnDagNode(obj)
        print(f"{fnDagNode.name()} : {fnDagNode.typeName}")
        if fnDagNode.parentCount() > 0:
            obj = fnDagNode.parent(0)
        else:
            return


printParent(trans2)

world : dagNode


In [67]:
def printFnOfObject(obj: om.MObject):
    for attr_name, attr_value in om.MFn.__dict__.items():
        if attr_name.startswith("k"):
            if obj.hasFn(attr_value):
                print(attr_name, attr_value)


printFnOfObject(trans1)

kBase 1
kNamedObject 2
kDependencyNode 4
kDagNode 107
kTransform 110


In [91]:
def printBaseClass(obj):
    print(obj.__class__)
    parent = obj.__class__.__bases__[0]
    while len(parent.__bases__) > 0:
        print(parent)
        parent = parent.__bases__[0]


printBaseClass(fnTrans)  # a MFnTransform

<class 'OpenMaya.MFnTransform'>
<class 'OpenMaya.MFnDagNode'>
<class 'OpenMaya.MFnDependencyNode'>
<class 'OpenMaya.MFnBase'>


In [95]:
SelectionList: om.MSelectionList = om.MGlobal.getActiveSelectionList()
print(SelectionList)

('hair_0')


In [147]:
dagPath: om.MDagPath = SelectionList.getDagPath(0)
node = SelectionList.getDependNode(0)

In [143]:
print(dagPath)

hair_0


In [268]:
printFnOfObject(node)

kBase 1
kNamedObject 2
kDependencyNode 4
kDagNode 107
kShape 248
kGeometric 265
kSurface 293
kMesh 296


In [365]:
SelectionList: om.MSelectionList = om.MGlobal.getActiveSelectionList()
# dagPath : om.MDagPath = SelectionList.getDagPath(0)
node = SelectionList.getDependNode(0)
fnDepNode = om.MFnDependencyNode(node)
# print(fnDepNode.userNode()) # MPxNode
print(fnDepNode.name() + " : " + fnDepNode.typeName)
printParent(node)


polyQuad4 : polyQuad
polyQuad4 is not a dag node


In [255]:
def getNodeByName(node_name) -> om.MObject | None:
    sel_list = om.MSelectionList()
    try:
        sel_list.add(node_name)
    except:
        return None
    return sel_list.getDependNode(0)


print(getNodeByName("pPlane1_rotateY"))

In [270]:
SelectionList

('pSphereShape1', 'pSphereShape1_tangentSmoothingAngle')


In [301]:
printFnOfObject(fnDepNode.attribute("height"))

kBase 1
kAttribute 565
kUnitAttribute 573


In [305]:
# height = om.MFnUnitAttribute(fnDepNode.attribute("height"))
heightPlug: om.MPlug = fnDepNode.findPlug(fnDepNode.attribute("height"), False)  # 这个bool值并没有对使用什么影响，应该只是内部实现的差异
print(heightPlug.attribute() == fnDepNode.attribute("height"))
heightPlug.setFloat(100.0)
print(heightPlug.asFloat())



True
100.0


In [347]:
def printPlugConnect(plug: om.MPlug):
    if plug.isConnected:
        if plug.isSource:
            print("-->", plug.destinations()[0], end="")
        elif plug.isDestination:
            print("<--", plug.source(), end="")


def printPlugs(fnDepNode: om.MFnDependencyNode):
    for i in range(fnDepNode.attributeCount()):
        attr: om.MFnAttribute = fnDepNode.attribute(i)
        plug: om.MPlug = fnDepNode.findPlug(attr, False)
        print(plug.info, end=' ')
        if plug.isArray:
            print(': Array ', end='')
            if plug.numConnectedElements() > 0:
                for j in range(plug.numElements()):
                    element: om.MPlug = plug.elementByLogicalIndex(j)
                    if element.isConnected:
                        print()
                        print(element.info, end=' ')
                        printPlugConnect(element)
        if plug.isCompound:
            print(': Compound ', end='')
        else:
            printPlugConnect(plug)
        print()


printPlugs(fnDepNode)

pCubeShape1.message 
pCubeShape1.caching 
pCubeShape1.frozen 
pCubeShape1.isHistoricallyInteresting 
pCubeShape1.nodeState 
pCubeShape1.binMembership 
pCubeShape1.hyperLayout 
pCubeShape1.isCollapsed 
pCubeShape1.blackBox 
pCubeShape1.borderConnections : Array 
pCubeShape1.isHierarchicalConnection : Array 
pCubeShape1.publishedNodeInfo : Array : Compound 
pCubeShape1.publishedNodeInfo[-1].publishedNode 
pCubeShape1.publishedNodeInfo[-1].isHierarchicalNode 
pCubeShape1.publishedNodeInfo[-1].publishedNodeType 
pCubeShape1.rmbCommand 
pCubeShape1.templateName 
pCubeShape1.templatePath 
pCubeShape1.viewName 
pCubeShape1.iconName 
pCubeShape1.viewMode 
pCubeShape1.templateVersion 
pCubeShape1.uiTreatment 
pCubeShape1.customTreatment 
pCubeShape1.creator 
pCubeShape1.creationDate 
pCubeShape1.containerType 
pCubeShape1.boundingBox : Compound 
pCubeShape1.boundingBoxMin : Compound 
pCubeShape1.boundingBoxMinX 
pCubeShape1.boundingBoxMinY 
pCubeShape1.boundingBoxMinZ 
pCubeShape1.boundingBoxMa

In [348]:
heightPlug.node()

In [349]:
itDag = om.MItDag()
# find mesh
itDag.reset(fnDepNode.object(), om.MItDag.kDepthFirst, om.MFn.kMesh)
while not itDag.isDone():
    mesh = om.MFnMesh(itDag.currentItem())
    print(mesh.name())
    itDag.next()

pCubeShape1


In [360]:
itDg = om.MItDependencyGraph(fnDepNode.object(), direction=om.MItDependencyGraph.kUpstream) # 选择遍历方向是按Plug连接的上游还是下游
# itDg.resetTo(fnDepNode.object())
while not itDg.isDone():
    dn = om.MFnDependencyNode(itDg.currentNode())
    if dn.typeName == 'xgmSplineBase':
        print(dn.name())
        break
    itDg.next()

description3_splineBase
